## Выпускной проект — дополнительная развлекательная программа

Вам поручили изучить характеристики процесса обработки стали.  
**Для каждого значения столбца `key` найдите**:
<a href='#section1'></a>    
1. [Длительность времени между первым и последним замером температуры;](#section1)
<a href='#section2'></a>    
2. [Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов;](#section1)
<a href='#section3'></a>    
3. [Количество запусков нагрева электродами;](#section1)
<a href='#section4'></a>    
4. [Среднее соотношение потребления активной и реактивной мощности.](#section1)

*По всем полученным столбцам вычислите статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей.*

In [122]:
import numpy as np
import pandas as pd

Загрузка данных

In [123]:
data_arc = pd.read_csv('/datasets/final_steel/data_arc.csv')
data_bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')
data_bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')
data_gas = pd.read_csv('/datasets/final_steel/data_gas.csv')
data_temp = pd.read_csv('/datasets/final_steel/data_temp.csv')
data_wire = pd.read_csv('/datasets/final_steel/data_wire.csv')
data_wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv')

In [124]:
data_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397


______

In [125]:
data_arc["Начало нагрева дугой"] = pd.to_datetime(data_arc["Начало нагрева дугой"])
data_arc["Конец нагрева дугой"] =  pd.to_datetime(data_arc["Конец нагрева дугой"])
data_arc["Время нагрева дугой"] = (data_arc["Конец нагрева дугой"] - data_arc["Начало нагрева дугой"]).dt.seconds
data_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность,Время нагрева дугой
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084,228
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285,185
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805,172
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669,365
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397,148


<a id='section2'></a>
### Суммарное время нагрева электродами

In [126]:
sum_heating = data_arc.groupby("key").sum()
sum_heating.head()

,Активная мощность,Реактивная мощность,Время нагрева дугой
key,,,
1,4.878147,3.183241,1098
2,3.052598,1.998112,811
3,2.525882,1.599076,655
4,3.209250,2.060298,741
5,3.347173,2.252643,869


In [136]:
sum_heating.describe()["Время нагрева дугой"]

count    3214.000000
mean      794.579029
std       332.448622
min        57.000000
25%       571.000000
50%       770.000000
75%       983.000000
max      4189.000000
Name: Время нагрева дугой, dtype: float64

<a id='section1'></a>
### Длительность времени между первым и последним замером температуры

In [127]:
data_temp["Время замера"] = pd.to_datetime(data_temp["Время замера"])
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15907 entries, 0 to 15906
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   key           15907 non-null  int64         
 1   Время замера  15907 non-null  datetime64[ns]
 2   Температура   13006 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 372.9 KB


In [128]:
temp_group = data_temp.groupby("key")
temp_timedelta_key = temp_group.max() - temp_group.min()
temp_timedelta_key.head()

,Время замера,Температура
key,,
1,0 days 00:14:21,47.0
2,0 days 00:21:45,31.0
3,0 days 00:21:40,3.0
4,0 days 00:06:28,24.0
5,0 days 00:12:42,26.0


In [139]:
temp_timedelta_key.describe()["Время замера"]

count                         3216
mean     0 days 00:30:39.861629353
std      0 days 00:20:46.950298194
min                0 days 00:00:00
25%                0 days 00:19:37
50%                0 days 00:26:21
75%                0 days 00:37:15
max                0 days 06:32:17
Name: Время замера, dtype: object

<a id='section3'></a>
### Количество запусков нагрева электродами

In [129]:
data_arc_keycounter = data_arc["key"].value_counts().sort_index()
data_arc_keycounter.head()

1    5
2    4
3    5
4    4
5    4
Name: key, dtype: int64

In [140]:
data_arc_keycounter.describe()

count    3214.000000
mean        4.628500
std         1.608647
min         1.000000
25%         4.000000
50%         4.000000
75%         6.000000
max        16.000000
Name: key, dtype: float64

<a id='section4'></a>
### Среднее соотношение потребления активной и реактивной мощности

In [130]:
def full_power(x): return np.sqrt(x["Реактивная мощность"]**2 + x["Активная мощность"]**2)

In [131]:
data_arc["Полная мощность"] = data_arc.apply(full_power, axis=1)
data_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность,Время нагрева дугой,Полная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084,228,1.193639
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285,185,0.959010
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805,172,0.896038
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669,365,1.970470
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397,148,0.807239


In [132]:
data_arc_power_ratio = data_arc[["key", "Активная мощность", "Реактивная мощность", "Полная мощность"]].groupby("key").sum()
data_arc_power_ratio["ratio_Активная мощность"] = data_arc_power_ratio["Активная мощность"] / data_arc_power_ratio["Полная мощность"]
data_arc_power_ratio["ratio_Реактивная мощность"] = data_arc_power_ratio["Реактивная мощность"] / data_arc_power_ratio["Полная мощность"]
data_arc_power_ratio.drop(["Активная мощность", "Реактивная мощность"], axis=1, inplace=True)
data_arc_power_ratio.head()

,Полная мощность,ratio_Активная мощность,ratio_Реактивная мощность
key,,,
1,5.826395,0.837250,0.546348
2,3.649216,0.836508,0.547545
3,2.989534,0.844908,0.534891
4,3.814962,0.841227,0.540057
5,4.035758,0.829379,0.558171


In [141]:
data_arc_power_ratio.describe()[["ratio_Активная мощность", "ratio_Реактивная мощность"]]

,ratio_Активная мощность,ratio_Реактивная мощность
count,3214.000000,3214.000000
mean,0.798655,0.597412
std,0.036461,0.048942
min,0.002578,-0.996450
25%,0.786767,0.573045
50%,0.804649,0.592633
75%,0.818894,0.616036
max,0.870471,0.847947
